In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
import prometheux_chain as pmtx

In [2]:
import os

PMTX_TOKEN = ""
os.environ["PMTX_TOKEN"] = PMTX_TOKEN

In [3]:
pmtx.config.set("LLM_PROVIDER", "openai")
pmtx.config.set("LLM_API_KEY", "")
pmtx.config.set("LLM_VERSION", "gpt-4o")
pmtx.config.set("LLM_TEMPERATURE", 0.5)
pmtx.config.set("LLM_MAX_TOKENS", 4096)

pmtx.config.set("EMBEDDING_PROVIDER", "openai")
pmtx.config.set("EMBEDDING_API_KEY", "")
pmtx.config.set("EMBEDDING_VERSION", "text-embedding-3-large")
pmtx.config.set("EMBEDDING_DIMENSIONS", 2048)

In [ ]:
pmtx.cleanup_projects()

In [ ]:
project_id = pmtx.save_project(project_name="test_project")
project_id

In [ ]:
pmtx.list_projects()

In [ ]:
project = pmtx.load_project(project_id=project_id)
project_id = project.get('id')
project_scope = project.get('scope')
project

In [ ]:
pmtx.cleanup_sources(project_id=project_id, project_scope=project_scope)

In [ ]:
db = pmtx.model.database.Database(
    database_type="csv",
    username="",
    password="",
    host="/Users/teodorobaldazzi/Documents/workdir/prometheux/prometheux_chain/test/resources/ownership",
    port=0000,
    database_name="people.csv",
    options={} #{"delimiter": "\t"}
)
pmtx.connect_sources(project_id=project_id, project_scope=project_scope, database_payload=db, add_model=True)

db = pmtx.model.database.Database(
    database_type="csv",
    username="",
    password="",
    host="/Users/teodorobaldazzi/Documents/workdir/prometheux/prometheux_chain/test/resources/ownership",
    port=0000,
    database_name="ownership.csv",
    options={} #{"delimiter": "\t"}
)
pmtx.connect_sources(project_id=project_id, project_scope=project_scope, database_payload=db, add_model=True)

db = pmtx.model.database.Database(
    database_type="csv",
    username="",
    password="",
    host="/Users/teodorobaldazzi/Documents/workdir/prometheux/prometheux_chain/test/resources/ownership",
    port=0000,
    database_name="companies.csv",
    options={} #{"delimiter": "\t"}
)
pmtx.connect_sources(project_id=project_id, project_scope=project_scope, database_payload=db, add_model=True)


# db = pmtx.model.database.Database(
#     database_type="neo4j",
#     username="neo4j",
#     password="",
#     host="neo4j.het.io",
#     port=7687,
#     database_name="neo4j"
# )
# pmtx.connect_sources(project_id=project_id, project_scope=project_scope, database_payload=db, add_model=True)

In [ ]:
project_sources = pmtx.list_sources(project_id=project_id, project_scope=project_scope)
project_sources

In [ ]:
pmtx.cleanup_sources(project_id=project_id, project_scope=project_scope, source_ids=[project_sources[0]['id']])

In [ ]:
pmtx.list_sources(project_id=project_id, project_scope=project_scope)

In [ ]:
pmtx.list_concepts(project_id=project_id, project_scope=project_scope)

In [ ]:
pmtx.cleanup_notebooks(project_id=project_id, project_scope=project_scope)

In [ ]:
notebook_id = pmtx.save_notebook(project_id=project_id, project_scope=project_scope, notebook_name="test_notebook")
notebook_id

In [ ]:
pmtx.load_notebook(project_id=project_id, notebook_id=notebook_id, project_scope=project_scope)

In [ ]:
pmtx.list_notebooks(project_id=project_id, project_scope=project_scope)

In [ ]:
pmtx.cleanup_cells(project_id=project_id, notebook_id=notebook_id, project_scope=project_scope)

In [ ]:
cell_content = """
test1(Name, Age) :- people_csv(Name, Age).
@output("test1").
"""
pmtx.run_cell(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id, cell_content=cell_content, cell_position=1)

In [ ]:
cell_content = """
test2(Owner, Company) :- ownership_csv(Owner, Company, Share).
@output("test2").
"""
cell_id = pmtx.save_cell(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id, cell_content=cell_content, cell_position=2)
cell_id

In [ ]:
cell_content = """
test2(Owner, Company) :- ownership_csv(Owner, Company, Share).
@output("test2").
"""
pmtx.run_cell(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id, cell_content=cell_content, cell_position=2, cell_id=cell_id)

In [ ]:
cell_content = """
personOfInterest("Emily Green").
personOfInterest("Daniel Black").
personOfInterest("Alice Johnson").
test3(Name, Company) :- test1(Name, Age), personOfInterest(Name), test2(Name, Company).
@output("test3").
"""
pmtx.run_cell(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id, cell_content=cell_content, cell_position=3)

In [ ]:
project_notebook_cells = pmtx.list_cells(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id)
project_notebook_cells

In [ ]:
pmtx.list_cell_outputs(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id, cell_id=project_notebook_cells[2]['id'], output_predicate="test3")

In [ ]:
pmtx.list_cell_outputs(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id, cell_id=project_notebook_cells[2]['id'], output_predicate="test3", page=2)

In [26]:
# pmtx.cleanup_cells(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id, cell_ids=[project_notebook_cells[2]['id']])

In [ ]:
pmtx.list_cells(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id)

In [ ]:
cell_content = """
test4(Name, Age, Company) :- test1(Name, Age), test3(Name, Company).
@output("test4").
"""
pmtx.save_cell(project_id=project_id, project_scope=project_scope, notebook_id=notebook_id, cell_content=cell_content, cell_position=4)

In [ ]:
pmtx.load_notebook(project_id=project_id, notebook_id=notebook_id, project_scope=project_scope)

In [ ]:
pmtx.cleanup_notebooks(project_id=project_id, project_scope=project_scope, notebook_ids=[notebook_id])

In [ ]:
pmtx.list_concepts(project_id=project_id, project_scope=project_scope)

In [ ]:
pmtx.cleanup_kgs(project_id=project_id, project_scope=project_scope)

In [ ]:
kg_id = pmtx.save_kg(project_id=project_id, project_scope=project_scope, kg_name="test_kg", kg_concepts=["test1","test2","test4"])

In [ ]:
kg_id = pmtx.build_kg(project_id=project_id, project_scope=project_scope, kg_id=kg_id, kg_name="test_kg", kg_concepts=["test1","test2","test4"], to_persist=True)

In [ ]:
kg = pmtx.load_kg(project_id=project_id, project_scope=project_scope, kg_id=kg_id)

In [ ]:
pmtx.visualize_kg_concepts(project_id=project_id, project_scope=project_scope, kg_id=kg_id)

In [ ]:
pmtx.cleanup_kg_cells(project_id=project_id, kg_id=kg_id, project_scope=project_scope)

In [ ]:
cell_content = """
test5(Company) :- test4(Name, Age, Company).
@output("test5").
"""
kg_cell_id = pmtx.save_kg_cell(project_id=project_id, project_scope=project_scope, kg_id=kg_id, cell_content=cell_content, cell_position=1)
kg_cell_id

In [ ]:
cell_content = """
test5(Company) :- test4(Name, Age, Company).
@output("test5").
"""
pmtx.run_kg_cell(project_id=project_id, project_scope=project_scope, kg_id=kg_id, cell_content=cell_content, cell_position=1, cell_id=kg_cell_id)

In [ ]:
project_kg_cells = pmtx.list_kg_cells(project_id=project_id, project_scope=project_scope, kg_id=kg_id)
project_kg_cells

In [ ]:
kg = pmtx.load_kg(project_id=project_id, project_scope=project_scope, kg_id=kg_id)

In [ ]:
pmtx.list_kg_cell_outputs(project_id=project_id, project_scope=project_scope, kg_id=kg_id, cell_id=project_kg_cells[0]['id'], output_predicate="test5", page=1)

In [ ]:
pmtx.cleanup_kg_cells(project_id=project_id, project_scope=project_scope, kg_id=kg_id, cell_ids=[project_kg_cells[0]['id']])

In [ ]:
project_kg_cells = pmtx.list_kg_cells(project_id=project_id, project_scope=project_scope, kg_id=kg_id)
project_kg_cells

In [ ]:
pmtx.cleanup_kg_chat(project_id=project_id, project_scope=project_scope, kg_id=kg_id)

In [ ]:
chat_id = pmtx.run_kg_chat(project_id=project_id, project_scope=project_scope, kg_id=kg_id, question="What are the companies of John Doe?", prompt_tuning="Reply in Italian").get('id')

In [ ]:
pmtx.list_kg_chat(project_id=project_id, project_scope=project_scope, kg_id=kg_id)

In [ ]:
pmtx.cleanup_kg_chat(project_id=project_id, project_scope=project_scope, kg_id=kg_id, chat_ids=[chat_id])

In [ ]:
pmtx.list_kg_chat(project_id=project_id, project_scope=project_scope, kg_id=kg_id)